# Smart Contract Crash Course 

## Basic interaction with `geth`

The most common client lib to interface with ethereum execution clients (`geth`,`ganache`,...) is **web3**.
* [web3.js](https://github.com/ethereum/web3.js/)
    - node.js version
    - [API](https://web3js.readthedocs.io/)
* [web3py](https://github.com/ethereum/web3.py)
    - python version
    - as I prefer python, we will use this one ;)
    - [API](https://web3py.readthedocs.io/)
    
For this tutorial we will only interact with `geth`: 
* [geth console](https://geth.ethereum.org/docs/interface/javascript-console) has a link to the supported API via the geth console, which should be the "full" `web3.js` API.
* [geth cli options](https://geth.ethereum.org/docs/interface/command-line-options) only needed when running geth from the command line

## Connect via RPC/HTTP

Note that in our case `geth` is running in a private PoA chain setup.
Therefore, some configuration parameters are different. 

https://web3py.readthedocs.io/en/stable/middleware.html#geth-style-proof-of-authority

For the challenge environment configuration the docker container for the geth client `Bob` is located at `172.18.0.8`.

In [1]:
import web3
from web3.middleware import geth_poa_middleware

w3 = web3.Web3(web3.Web3.HTTPProvider("http://geth-client-cnt:8545"))
# check if connection was successful
assert w3.is_connected()

w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [2]:
w3.geth.admin.node_info()

AttributeDict({'id': '76a8c5bca218e0ecfca09818ff4112faaf6aa1c9af9947ef3518e12127fa2b1b',
 'name': 'Geth/bob/v1.10.3-stable-991384a7/linux-amd64/go1.21.5',
 'enode': 'enode://d8ed6c277b0878cad10cc7a53daa8efbfe4a04b73016a484ef1df3eca7bd44d3d06c0707030ec5fbfcad0f8350e2a79a8491f5b6379e9cf4f5fcd04e8c6fa03d@127.0.0.1:30303?discport=0',
 'enr': 'enr:-Ja4QGjX13WHXG9FOVyLyObNbiN3-ZbAwp4ECD9rmp5J-CHySk0NqpsOZMf9XGgvvJvq6IDNeqA4u3O_csPcvlAdCQoCg2V0aMfGhL8tJ2SAgmlkgnY0gmlwhH8AAAGJc2VjcDI1NmsxoQPY7Wwnewh4ytEMx6U9qo77_koEtzAWpITvHfPsp71E04RzbmFwwIN0Y3CCdl8',
 'ip': '127.0.0.1',
 'ports': AttributeDict({'discovery': 0, 'listener': 30303}),
 'listenAddr': '[::]:30303',
 'protocols': AttributeDict({'eth': AttributeDict({'network': 20240101,
   'difficulty': 1,
   'genesis': '0x6bec1279ca6efde179db76897011152e3b1d4c89f11298b501a501511fa016b8',
   'config': AttributeDict({'chainId': 20240101,
    'homesteadBlock': 0,
    'eip150Block': 0,
    'eip150Hash': '0x000000000000000000000000000000000000000000000

In [3]:
# display the client version of the node we are connected to
w3.client_version

'Geth/bob/v1.10.3-stable-991384a7/linux-amd64/go1.21.5'

In [4]:
# display network ID of client you are connected to
w3.net.version

'20240101'

In [5]:
w3.eth.chain_id

20240101

In [6]:
# Display number of connected peers should be 1
w3.net.peer_count

0

In [7]:
# check if node is "up-to-date" with blockchain, 
# i.e., not syncing anymore
w3.eth.syncing

False

In [8]:
w3.eth.mining

False

## Connect to enode with IP

To connect to a geth node a `enode` ulr is required. This contains a public key of this node as well as its domain name (or IP) and prot. 

The enode of our server is: 
`enode://fd90f39e40633934f1613ab08bd4917d3aa28abba28cc4748b544650cfffe1ab40890f4a9d85ca9c0cf28845633291ca06a5716b86f5232b32374b82e2e3cff6@eth-smart.secenv:30303?discport=0`

In [9]:
enode = "enode://fd90f39e40633934f1613ab08bd4917d3aa28abba28cc4748b544650cfffe1ab40890f4a9d85ca9c0cf28845633291ca06a5716b86f5232b32374b82e2e3cff6@eth-smart.secenv:30303?discport=0"

In [10]:
#enode = "enode://fd90f39e40633934f1613ab08bd4917d3aa28abba28cc4748b544650cfffe1ab40890f4a9d85ca9c0cf28845633291ca06a5716b86f5232b32374b82e2e3cff6@10.81.0.38:30303?discport=0"

In [11]:
w3.geth.admin.add_peer(enode)

True

In [12]:
w3.net.peer_count

1

In [13]:
w3.geth.admin.peers()

[AttributeDict({'enode': 'enode://fd90f39e40633934f1613ab08bd4917d3aa28abba28cc4748b544650cfffe1ab40890f4a9d85ca9c0cf28845633291ca06a5716b86f5232b32374b82e2e3cff6@10.81.0.38:30303?discport=0',
  'id': '2af0d687348cffa0a8d8e827abd4e1e5f2c68e4beca2dfaa768856a9dbba522e',
  'name': 'Geth/alice/v1.10.3-stable-991384a7/linux-amd64/go1.21.5',
  'caps': ['eth/65', 'eth/66', 'snap/1'],
  'network': AttributeDict({'localAddress': '172.22.0.4:45678',
   'remoteAddress': '10.81.0.38:30303',
   'inbound': False,
   'trusted': False,
   'static': True}),
  'protocols': AttributeDict({'eth': AttributeDict({'version': 66,
    'difficulty': 721,
    'head': '0x541ae28b8298d48d07b8ff02156e18c45756aa7522ed789b6302087d187cae47'}),
   'snap': AttributeDict({'version': 1})})})]

To manually extract the enode from a running geth node, cat the `nodekey` file which contains the private key  and use the `bootnode` tool to generate the associated public key, i.e., enode. (see )

```bash
$ cat $DATADIR/geth/nodekey
$ bootnode -nodekeyhex $(cat $DATADIR/geth/nodekey) -writeaddress
```

In [14]:
w3.eth.syncing

AttributeDict({'currentBlock': 0,
 'highestBlock': 360,
 'knownStates': 1,
 'pulledStates': 0,
 'startingBlock': 0})

## Blocks

In [51]:
# return current blockchain head of node
w3.eth.block_number

0

In [48]:
# should be greater
!sleep 16
w3.eth.block_number

0

In [20]:
# block 0 ist the genesis block of this testnet blockchain
genesis_block = w3.eth.get_block(0)
genesis_block

AttributeDict({'difficulty': 1,
 'proofOfAuthorityData': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000f78342e0cdb2188e3b5c9663b1c6d6d5af68bc970000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'gasLimit': 15000000,
 'gasUsed': 0,
 'hash': HexBytes('0x6bec1279ca6efde179db76897011152e3b1d4c89f11298b501a501511fa016b8'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'miner': '0x00000000000

As you see, the genesis block is special as it is the only block with no parent hash:

In [21]:
genesis_block['number']

0

In [22]:
genesis_block['parentHash']

HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000')

The [genesis.json](../genesis_config/go-ethereum/berlin/genesis.json) file shows the node(s) that are allowed to mine blocks and the accounts that have been seeded with coins. 
The client we are currently connected to is not part of the PoA nodes that are allowed to create new blocks:

## Accounts and balances

Accounts are your **external accounts** consisting of public and private keys. 
The ethereum address are the last 40 hex characters (20 bytes) of the hash (Keccak-256) of the public key prefixed with `0x`

For the exercise you will recieve your personal account keypair which you have to copy in your `geth` folder to import it. 

In [29]:
!pwd

/smartenv


In [31]:
!ls 

Containerfile  requirements.txt  sccc.ipynb  venv
entrypoint.sh  sccc-anvil.ipynb  test.ipynb


In [25]:
DATADIR_PATH = "/smartenv/datadir/"

In [30]:
!ls {DATADIR_PATH}bob/keystore

ls: cannot access '/smartenv/datadir/bob/keystore': No such file or directory


In [32]:
w3.eth.accounts

['0xe884e188beD351dC094105b22C9752157656a57b']

The default address (also used to receive rewards from mining blocks) is usually the first in the list of accounts and usually also accessible via `w3.eth.coinbase`. It is good practise to specifically assing a `default_account` some functions might cause problems if this is not explicitly assigned.

In [51]:
assert w3.eth.accounts[0] == w3.eth.coinbase
w3.eth.default_account = w3.eth.accounts[0]
w3.eth.default_account 

'0xa1273F73C607Bd0af4D2916f4C9e6A550581dCA6'

In [52]:
def getBalance(address):
    return w3.fromWei(w3.eth.getBalance(address),'ether')

In [55]:
initial_balance = getBalance(w3.eth.accounts[0]) 
assert initial_balance > 0 
initial_balance

Decimal('999998.999979')

### Create New Account

Create a new account with no password:

In [56]:
!geth --maxpeers 0 --datadir={DATADIR_PATH}/bob/ --password=/dev/null account new

INFO [06-16|12:08:45.091] Maximum peer count                       ETH=0 LES=0 total=0
INFO [06-16|12:08:45.091] Smartcard socket not found, disabling    err="stat /run/pcscd/pcscd.comm: no such file or directory"

Your new key was generated

Public address of the key:   0x782FEdd38Bd647fcb825cEDb39Af0b2ee27cA49E
Path of the secret key file: /smartenv/datadir/bob/keystore/UTC--2021-06-16T10-08-45.091849684Z--782fedd38bd647fcb825cedb39af0b2ee27ca49e

- You can share your public address with anyone. Others need it to interact with you.
- You must NEVER share the secret key with anyone! The key controls access to your funds!
- You must BACKUP your key file! Without the key, it's impossible to access account funds!
- You must REMEMBER your password! Without the password, it's impossible to decrypt the key!



It requires some time till geth picks up the new key:

In [57]:
!sleep 5

In [58]:
w3.eth.accounts

['0xa1273F73C607Bd0af4D2916f4C9e6A550581dCA6',
 '0x782FEdd38Bd647fcb825cEDb39Af0b2ee27cA49E']

## Payment Transaction and Gas

**Note:** When connected to `geth` or `parity` you have to unlock an account first to send a transaction since the private keys are stored encrypted per default and protected with a password. In `ganache` this is not necessary since it is a development environment.

Per default unlocking an account via HTTP is not allowed in `geth` so this has to be explicitly activated 
with `--allow-insecure-unlock` when starting `geth`. Do not use this flag in production! 

In [59]:
# Parameters are:
# * account
# * password
# * duration of unlock (if 0 then forever)
w3.geth.personal.unlockAccount(w3.eth.accounts[0],"notforproductiveuse",0)

True

In [60]:
blkNumber_before = w3.eth.blockNumber
blkNumber_before

42590

In [61]:
w3.eth.sendTransaction({'from':w3.eth.accounts[0], 
                        'to':w3.eth.accounts[1], 
                        'value':10**18})

HexBytes('0x7961f9ac7455bab5fbd34d409e055f3066b5ad5041cdeb1af5e536dbad7aa6bc')

In [62]:
!sleep 16
blkNumber_after = w3.eth.blockNumber
assert blkNumber_before < blkNumber_after
blkNumber_after

42591

In [63]:
# the balance is reduced by more than 1 ether
# Why do you think this is the case?
getBalance(w3.eth.accounts[0])

Decimal('999997.999958')

In [69]:
new_account_balance = getBalance(w3.eth.accounts[1])
assert int(new_account_balance) == 1
int(new_account_balance)

1

In [70]:
# get first (and only) transaction in the previously mined block
tx = w3.eth.getTransactionByBlock(blkNumber_before + 1,0)
tx 

AttributeDict({'blockHash': HexBytes('0x63bd35ec9cd40da2800e2b9593c5aa0af514d13bf436ca716e5674d4dec7a7f5'),
 'blockNumber': 42591,
 'from': '0xa1273F73C607Bd0af4D2916f4C9e6A550581dCA6',
 'gas': 121000,
 'gasPrice': 1000000000,
 'hash': HexBytes('0x7961f9ac7455bab5fbd34d409e055f3066b5ad5041cdeb1af5e536dbad7aa6bc'),
 'input': '0x',
 'nonce': 1,
 'to': '0x782FEdd38Bd647fcb825cEDb39Af0b2ee27cA49E',
 'transactionIndex': 0,
 'value': 1000000000000000000,
 'type': '0x0',
 'v': 2709,
 'r': HexBytes('0x4e28aaa8cca628b7cddf041d887642195b152def3a9ea1bd96a1e3872dcb269f'),
 's': HexBytes('0x3c0bafa0b1617a782bd8b0fc63a8da8664f3dd869fa8b6237c9110fbd8857177')})

In [71]:
# get block and see the transaction id and gas used below
blk = w3.eth.getBlock(blkNumber_before + 1)
blk

AttributeDict({'difficulty': 2,
 'proofOfAuthorityData': HexBytes('0xd883010a04846765746888676f312e31352e34856c696e757800000000000000eb2331c32c7c3a1e71735f0584b41e098872549acd3b79fb61adb6f013e8d2ca73d57fa89314c06e9f121c3fc1877ec8fdd5f25209536f5efbad099e510a568901'),
 'gasLimit': 8000000,
 'gasUsed': 21000,
 'hash': HexBytes('0x63bd35ec9cd40da2800e2b9593c5aa0af514d13bf436ca716e5674d4dec7a7f5'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'miner': '0x0000000000000000000000000000000000000000',
 'mix

In [72]:
# get the transaction ID of the first (and only) transaction in the block
blk["transactions"][0].hex()

'0x7961f9ac7455bab5fbd34d409e055f3066b5ad5041cdeb1af5e536dbad7aa6bc'

In [73]:
tx["hash"].hex() # hash of the transaction

'0x7961f9ac7455bab5fbd34d409e055f3066b5ad5041cdeb1af5e536dbad7aa6bc'

In [74]:
# Query the transaction by ID, also possible
w3.eth.getTransaction(blk["transactions"][0].hex())

AttributeDict({'blockHash': HexBytes('0x63bd35ec9cd40da2800e2b9593c5aa0af514d13bf436ca716e5674d4dec7a7f5'),
 'blockNumber': 42591,
 'from': '0xa1273F73C607Bd0af4D2916f4C9e6A550581dCA6',
 'gas': 121000,
 'gasPrice': 1000000000,
 'hash': HexBytes('0x7961f9ac7455bab5fbd34d409e055f3066b5ad5041cdeb1af5e536dbad7aa6bc'),
 'input': '0x',
 'nonce': 1,
 'to': '0x782FEdd38Bd647fcb825cEDb39Af0b2ee27cA49E',
 'transactionIndex': 0,
 'value': 1000000000000000000,
 'type': '0x0',
 'v': 2709,
 'r': HexBytes('0x4e28aaa8cca628b7cddf041d887642195b152def3a9ea1bd96a1e3872dcb269f'),
 's': HexBytes('0x3c0bafa0b1617a782bd8b0fc63a8da8664f3dd869fa8b6237c9110fbd8857177')})

In [75]:
gasUsed = blk["gasUsed"] # gas used within block
gasUsed

21000

In [76]:
# estimate the gas price of an transaction without executing it
w3.eth.estimateGas({'from':w3.eth.accounts[0], 
                    'to':w3.eth.accounts[1], 
                    'value':10**18})

21000

The estimate yields the same result as the actual gas cost. 
Can this be different?

In [77]:
gasPrice = tx["gasPrice"] # The actual gas price attached to the tx
gasPrice

1000000000

In [78]:
# conversion between gas and ether specified per transaction
# i.e., how much wei a transaction is willing to pay for 
assert gasPrice == w3.eth.gasPrice # default value

In [79]:
# gasUsed(by the transaction) * gasPrice(specified in tx)
# gives the wei it costs to send it 
# times 10^18 gives the ether
tx_costs = (gasUsed * gasPrice) * 10**-18
tx_costs 

2.1000000000000002e-05

This is exaclty the missing cost on the sender side: 

In [80]:
float(getBalance(w3.eth.accounts[0]))

999997.999958

In [81]:
float(getBalance(w3.eth.accounts[0])) + tx_costs

999997.9999790001

# TODO

In [2]:
'/challenges/notawallet/'

'/challenges/notawallet/'

In [1]:
!solc --version

solc, the solidity compiler commandline interface
Version: 0.8.23+commit.f704f362.Linux.g++


In [4]:
%pwd

'/smartenv'

In [5]:
import util

In [10]:
SMARTENV_PATH = "/smartenv/"
GEN_CONFIG_PATH = "/setup/generated_config/"
GLOBAL_INFO_PATH = GEN_CONFIG_PATH + "/global_info/"
GENESIS_JSON = "/setup/genesis_config/go-ethereum/berlin/genesis.json"

PARTICIPANTS_PATH = "/students/"
PARTICIPANTS_CSV = PARTICIPANTS_PATH + "list.csv"
PP_CSV = PARTICIPANTS_CSV

SCRIPTS_PATH = "/smartenv/util/scripts/"
CHALLENGES_PATH = "/challenges/"
CHL_NOTAWALLET_PATH = "/challenges/notawallet/"

In [11]:
SRVIP = "gethcnt"
SRVPORT = "8545"
SRVCHAINID = "20240101" # CHANGE THIS IS UPDATED!
SRVNETVER = SRVCHAINID

In [12]:
## Connections refresh maybe needed
w3=util.connect(host=SRVIP,port=SRVPORT,poa=True) 
assert w3.is_connected()

In [43]:
!cat {CHL_NOTAWALLET_PATH}publish/NotAWallet.sol

// SPDX-License-Identifier: MIT
/**

Look Mom, I coded my first wallet contract to learn solidity!
It even supports multiple owners!

**/

pragma solidity 0.8.19;

contract NotAWallet{

	address public owner;
    address public student;
    mapping (address => bool) owners;
   
    constructor(address _student) payable{
        student = _student;
        owner = msg.sender;
    }

    function deposit() public payable  { 
        // recieve coins
    }   

    function addOwner(address newowner) public {
        require (owners[msg.sender] == true || msg.sender == owner);
        owners[newowner] = true;
    }   

    function removeOwner(address oldowner) public rightStudent {
        require(owners[msg.sender] = true); 
				owners[oldowner] = false;
    }

    function withdraw(uint256 amount) public {
        // only owners can withdraw funds
        require(owners[msg.sender] == true);
        payable(msg.sender).transfer(amount);
    }

		function isOwner(address testowner) publi

In [46]:
!solc_8.19 --overwrite -o "{CHL_NOTAWALLET_PATH}/publish/" --abi --bin "{CHL_NOTAWALLET_PATH}/publish/NotAWallet.sol"

Compiler run successful. Artifact(s) can be found in directory "/challenges/notawallet//publish/".


In [47]:
!pwd

/smartenv


In [48]:
!ls {CHL_NOTAWALLET_PATH}publish

NotAWallet.abi	NotAWallet.bin	NotAWallet.sol	README.md


In [49]:
!cat {CHL_NOTAWALLET_PATH}publish/NotAWallet.abi

[{"inputs":[{"internalType":"address","name":"_student","type":"address"}],"stateMutability":"payable","type":"constructor"},{"inputs":[{"internalType":"address","name":"newowner","type":"address"}],"name":"addOwner","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getStudent","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"testowner","type":"address"}],"name":"isOwner","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"oldowner","type":"address"}],"name":"removeOwner","outputs":[],"stateMutability":"nonpayable","type":"functi

In [50]:
!cat {CHL_NOTAWALLET_PATH}publish/NotAWallet.bin

608060405260405161094d38038061094d8339818101604052810190610025919061010f565b80600160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055505061013c565b600080fd5b600073ffffffffffffffffffffffffffffffffffffffff82169050919050565b60006100dc826100b1565b9050919050565b6100ec816100d1565b81146100f757600080fd5b50565b600081519050610109816100e3565b92915050565b600060208284031215610125576101246100ac565b5b6000610133848285016100fa565b91505092915050565b6108028061014b6000396000f3fe60806040526004361061007b5760003560e01c80637065cb481161004e5780637065cb481461013a5780638da5cb5b14610163578063b439548d1461018e578063d0e30db0146101b95761007b565b8063173825d9146100805780632e1a7d4d146100a95780632f54bf6e146100d2578063699450e81461010f575b600080fd5b34801561008c57600080fd5b506100a760048036038101906100a2919061065f565b6101c3565b00

In [51]:
cbin = !cat {CHL_NOTAWALLET_PATH}publish/NotAWallet.bin

In [52]:
cabi = !cat {CHL_NOTAWALLET_PATH}publish/NotAWallet.abi

In [53]:
cabi = cabi[0]
cabi

'[{"inputs":[{"internalType":"address","name":"_student","type":"address"}],"stateMutability":"payable","type":"constructor"},{"inputs":[{"internalType":"address","name":"newowner","type":"address"}],"name":"addOwner","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getStudent","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"testowner","type":"address"}],"name":"isOwner","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"oldowner","type":"address"}],"name":"removeOwner","outputs":[],"stateMutability":"nonpayable","type":"funct

In [54]:
cbin = cbin[0]
cbin

'608060405260405161094d38038061094d8339818101604052810190610025919061010f565b80600160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055505061013c565b600080fd5b600073ffffffffffffffffffffffffffffffffffffffff82169050919050565b60006100dc826100b1565b9050919050565b6100ec816100d1565b81146100f757600080fd5b50565b600081519050610109816100e3565b92915050565b600060208284031215610125576101246100ac565b5b6000610133848285016100fa565b91505092915050565b6108028061014b6000396000f3fe60806040526004361061007b5760003560e01c80637065cb481161004e5780637065cb481461013a5780638da5cb5b14610163578063b439548d1461018e578063d0e30db0146101b95761007b565b8063173825d9146100805780632e1a7d4d146100a95780632f54bf6e146100d2578063699450e81461010f575b600080fd5b34801561008c57600080fd5b506100a760048036038101906100a2919061065f565b6101c3565b0

In [55]:
contract=w3.eth.contract(abi=cabi,
                         bytecode=cbin)
tx_hash = contract.constructor('0x53B30788b6a47261be56a851C22B155cd3b84735').transact({"from":w3.eth.accounts[0],})

In [56]:
tx_hash

HexBytes('0xfa5a9983cdd47795f7cab1f1b14e232b580d4c2e1ef5a90b6fff1106a7dd890f')

In [57]:
w3.eth.wait_for_transaction_receipt(tx_hash)

AttributeDict({'blockHash': HexBytes('0x0bb3124ae6de188b8e518056cc3bca99ffcd1b6b703e27d23fe1263c09c518a7'),
 'blockNumber': 1318,
 'contractAddress': '0xF5267338cBBBD0Bb48718c89E352976A9afA87dB',
 'cumulativeGasUsed': 544586,
 'from': '0xf78342E0cdb2188e3B5c9663b1C6D6d5AF68BC97',
 'gasUsed': 544586,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'status': 1,
 'to': None,
 'transactionHash': HexBytes('0xfa5a9983cdd47795f7cab1f1b14e232b580d4c2e1ef5a90b6fff1106a7dd890f'),
 'transactionInde

In [58]:
w3.eth.wait_for_transaction_receipt(tx_hash)

AttributeDict({'blockHash': HexBytes('0x0bb3124ae6de188b8e518056cc3bca99ffcd1b6b703e27d23fe1263c09c518a7'),
 'blockNumber': 1318,
 'contractAddress': '0xF5267338cBBBD0Bb48718c89E352976A9afA87dB',
 'cumulativeGasUsed': 544586,
 'from': '0xf78342E0cdb2188e3B5c9663b1C6D6d5AF68BC97',
 'gasUsed': 544586,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'status': 1,
 'to': None,
 'transactionHash': HexBytes('0xfa5a9983cdd47795f7cab1f1b14e232b580d4c2e1ef5a90b6fff1106a7dd890f'),
 'transactionInde